In [11]:
import regex as re
import os
from typing import List, Dict
import requests
import json

from pprint import pprint

GROQ_API_KEY = os.getenv('groq_key')

FILE_PATH = r'C:\Users\fuedj\Documents\Code\RAG_Dr_Voss\llm-case-study-main\src\parte1_another.txt'

In [12]:
def load_txt(file_path: str) -> str:
    """Carrega o conteúdo do arquivo de texto"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def chunk_text(text: str, chunk_size: int = 800) -> List[str]:
    """Divide o texto em chunks"""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

In [13]:
text = load_txt(FILE_PATH)


chunks = chunk_text(text)

In [14]:
len(text)

1432

In [15]:
len(chunks)

2

In [16]:
def split_large_chunk(chunk: str, max_size: int = 8000) -> list[str]:
    parts = []
    while len(chunk) > max_size:
        cut_index = chunk.rfind('.', 0, max_size)
        if cut_index == -1:
            cut_index = chunk.rfind(' ', 0, max_size)
        if cut_index == -1:
            cut_index = max_size  # Não achou ponto nem espaço, corta bruto
        parts.append(chunk[:cut_index+1].strip())
        chunk = chunk[cut_index+1:].strip()
    if chunk:
        parts.append(chunk)
    return parts

def chunk_diary_by_day_and_paragraph(diary_text: str) -> list[str]:
    chunks = []
    current_day_content = ""
    date_pattern = r"(\d{1,2})(?:st|nd|rd|th)? Day of ([A-Za-z]+) (18\d{2}) - ([A-Za-z\s]+)"

    for line in diary_text.splitlines():
        if re.match(date_pattern, line):
            # Encontrou um novo dia
            if current_day_content:
                # Divida o conteúdo do dia anterior por parágrafos (linhas não vazias)
                paragraphs = [p.strip() for p in current_day_content.strip().split('\n\n') if p.strip()]
                chunks.extend(paragraphs)
            current_day_content = line + "\n"  # Comece o conteúdo do novo dia
        else:
            current_day_content += line + "\n"

    # Processar o conteúdo do último dia
    if current_day_content:
        paragraphs = [p.strip() for p in current_day_content.strip().split('\n\n') if p.strip()]
        for paragraph in paragraphs:
            if len(paragraph) > 800:
                chunks.extend(split_large_chunk(paragraph))
            else:
                chunks.append(paragraph)
    
    return [chunk for chunk in chunks if chunk] # Remove empty ones 

list_chunk = chunk_diary_by_day_and_paragraph(text)

In [17]:
chunks = list_chunk
len(chunks)

3

In [ ]:
import aiohttp
import asyncio
import json
from typing import List, Dict, Optional, Tuple

class AgentSystem:
    def __init__(self, api_key: str, question: str):
        self.api_key = api_key
        self.partial_answers = []
        self.session = None
        self.question = question

    async def __aenter__(self):
        self.session = aiohttp.ClientSession()
        return self

    async def __aexit__(self, *args):
        await self.session.close()

    async def expert_agent(self, chunk: str) -> Optional[Dict]:
        """Agente especializado em análise narrativa"""
        prompt = f"""Analyze this text excerpt regarding:
        {self.question}

        Return a VALID JSON with these REQUIRED fields:
        {{
            "analysis": {{
                "narrative_elements": {{
                    "locations": list[str],
                    "interactions": list[str],
                    "methods": list[str]
                }},
                "regional_insights": {{
                    "veridia": list[str],
                    "eldoria": list[str]
                }}
            }},
            "metadata": {{
                "relevance_score": float (0-1),
                "confidence": float (0-1),
                "contains_key_insight": bool
            }}
        }}

        Text excerpt:
        {chunk[:1500]}"""

        try:
            async with self.session.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={"Authorization": f"Bearer {self.api_key}"},
                json={
                    "model": "qwen-qwq-32b",
                    "messages": [{
                        "role": "system",
                        "content": "You are a professional literary analyst. Return valid JSON."
                    }, {
                        "role": "user",
                        "content": prompt
                    }],
                    "temperature": 0.3,
                    "response_format": {"type": "json_object"}
                },
                timeout=20
            ) as response:
                data = await response.json()
                result = json.loads(data['choices'][0]['message']['content'])
                
                print(f"Expert Agent Response: {result}")
                
                # Verificação robusta da estrutura
                if all(key in result for key in ['analysis', 'metadata']):
                    if result['metadata'].get('relevance_score', 0) > 0.4:  # Threshold ajustável
                        self.partial_answers.append(result)
                        return result
                return None

        except json.JSONDecodeError as e:
            print(f"JSON Error in chunk analysis: {str(e)}")
            return None
        except KeyError as e:
            print(f"Missing key in response: {str(e)}")
            return None
        except Exception as e:
            print(f"Expert agent error: {str(e)}")
            return None

    async def central_agent(self) -> Dict:
        """Agente de síntese comparativa"""
        context = "\n".join(
            f"Chunk {i}:\n{json.dumps(ans, indent=2)}" 
            for i, ans in enumerate(self.partial_answers)
        )

        prompt = f"""Synthesize this analysis about:
        {self.question}

        Partial Analyses:
        {context}

        Return JSON with:
        - comparative_analysis: {{
            documentation_methods: {{
                veridia: list[str],
                eldoria: list[str],
                shared: list[str]
            }},
            cultural_insights: {{
                veridia: list[str],
                eldoria: list[str]
            }}
          }}
        - narrative_arc: {{
            key_events: list[str],
            character_development: list[str]
          }}
        - unanswered_questions: list[str]"""

        try:
            async with self.session.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={"Authorization": f"Bearer {self.api_key}"},
                json={
                    "model": "qwen-qwq-32b",
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": 0.1,
                    "response_format": {"type": "json_object"}
                },
                timeout=20
            ) as response:
                data = await response.json()
                
                print(f"Central Agent:")
                print(data['choices'][0]['message']['content'])
                
                return json.loads(data['choices'][0]['message']['content'])

        except Exception as e:
            print(f"Central agent error: {str(e)}")
            return {"status": "consolidation_error"}
        
    async def deep_analysis_agent(self, chunk: str):
        """Agente para análise qualitativa aprofundada"""
        prompt = f"""Perform deep qualitative analysis of this text regarding:
        {self.question}
        
        Focus on:
        - Narrative devices used
        - Cultural significance
        - Author's perspective shifts
        """
        
        
        try:
            async with self.session.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={"Authorization": f"Bearer {self.api_key}"},
                json={
                    "model": "qwen-qwq-32b",  # Modelo mais capaz para síntese
                    "messages": [{"role": "user", "content": prompt}],
                    "temperature": 0.1,
                    "response_format": {"type": "json_object"}
                },
                timeout=20
            ) as response:
                data = await response.json()
                return json.loads(data['choices'][0]['message']['content'])

        except Exception as e:
            print(f"Central agent error: {str(e)}")
            return {"status": "consolidation_error"}

async def analyze_document(api_key: str, question: str, chunks: List[str]) -> Tuple[Dict, List[Dict]]:
    """Processa o documento completo e retorna resposta consolidada + informações parciais"""
    async with AgentSystem(api_key, question=question) as system:
        # Primeira passada - Análise básica
        tasks = [system.expert_agent(chunk) for chunk in chunks]
        await asyncio.gather(*tasks)
        
        # # Segunda passada - Análise aprofundada nos chunks relevantes
        # deep_analysis_tasks = [
        #     system.deep_analysis_agent(chunk) 
        #     for chunk in chunks
        # ]
        # await asyncio.gather(*deep_analysis_tasks)
        
        return await system.central_agent()

# Exemplo de uso:
async def main():
    question = """
    
    Analyze the author's journey through Veridia and Eldoria, detailing the methods they employ
    to document their experiences, and discuss how their interactions with people
    and places shape their understanding of these regions' unique identities.
    """
    
    document_chunks = chunks  # Seus chunks aqui
    
    final_answer = await analyze_document(GROQ_API_KEY, question, document_chunks[:5])  # Teste com 5 chunks
    
    print("\n=== Resposta Consolidada ===")
    print(json.dumps(final_answer, indent=2, ensure_ascii=False))
    
    # print("\n=== Informações Parciais ===")
    # for i, partial in enumerate(partials, 1):
    #     print(f"\nFonte {i}:")
    #     print(json.dumps(partial, indent=2))

# Para executar no Jupyter:
await main()

Expert Agent Response: {'analysis': {'narrative_elements': {'locations': ['capital city of Veridia', 'Assembly House'], 'interactions': ['Observation of cultural artifacts (sculptures, paintings)', 'Attending legislative debate on sustainability initiatives'], 'methods': ['Journal entries', 'Documentation of public policy discussions', 'Visual documentation of art and architecture']}, 'regional_insights': {'veridia': ["Cultural emphasis on arts under Queen Isolde's patronage", 'Strong legislative focus on sustainability (80% renewable energy by 2050)', 'Blend of historical heritage with forward-thinking governance'], 'eldoria': []}}, 'metadata': {'relevance_score': 0.9, 'confidence': 0.85, 'contains_key_insight': True}}
Expert Agent Response: {'analysis': {'narrative_elements': {'locations': ['Veridia (Takron Valley)', 'Eldoria'], 'interactions': ["Engagement with a storyteller recounting Eldoria's Autumn festival", "Observation of Takron Valley's natural and labor environments"], 'met

In [18]:
chunks = chunks

In [17]:
len(chunks)

5

In [41]:
import aiohttp
import asyncio
import json
from typing import List, Dict, Optional, Tuple

class SmartAgentSystem:
    def __init__(self, api_key: str, question: str, all_chunks: List[str]):
        self.api_key = api_key
        self.question = question
        self.all_chunks = all_chunks
        self.processed_chunks = set()
        self.partial_answers = []
        self.session = None

    async def __aenter__(self):
        self.session = aiohttp.ClientSession()
        return self

    async def __aexit__(self, *args):
        await self.session.close()

    async def expert_agent(self, chunk: str, chunk_id: int) -> Optional[Dict]:
        """Agente especialista com identificação de chunk"""
        try:
            prompt = f"""Analyze this text regarding: {self.question}
            
            Required JSON format:
            {{
                "analysis": {{
                    "key_findings": ["list", "of", "findings"],
                    "missing_info": ["list", "of", "gaps"]
                }},
                "metadata": {{
                    "chunk_id": {chunk_id},
                    "relevance": 0.0-1.0,
                    "needs_more_context": true/false
                }}
            }}

            Text: {chunk[:2000]}"""

            async with self.session.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={"Authorization": f"Bearer {self.api_key}"},
                json={
                    "model": "qwen-qwq-32b",
                    "messages": [{
                        "role": "system",
                        "content": "You must respond in valid JSON format exactly as specified."
                    }, {
                        "role": "user",
                        "content": prompt
                    }],
                    "temperature": 0.1,
                    "response_format": {"type": "json_object"}
                },
                timeout=20
            ) as response:
                response.raise_for_status()
                data = await response.json()
                result = json.loads(data['choices'][0]['message']['content'])
                
                print(f"Experto Agent Decision: {result}")
                
                # Validate response structure
                required_keys = {'analysis', 'metadata'}
                if not all(key in result for key in required_keys):
                    raise ValueError("Missing required keys in response")
                
                self.processed_chunks.add(chunk_id)
                if result['metadata']['relevance'] > 0.4:
                    self.partial_answers.append(result)
                return result
            
        except json.JSONDecodeError as e:
            print(f"JSON Error in chunk {chunk_id}: {str(e)}")
        except KeyError as e:
            print(f"Missing key in chunk {chunk_id} response: {str(e)}")
        except Exception as e:
            print(f"Error processing chunk {chunk_id}: {str(e)}")
        return None

    async def central_agent(self) -> Dict:
        """Agente central que pode solicitar mais chunks"""
        max_attempts = 3
        for attempt in range(max_attempts):
            try:
                context = "\n".join(
                    f"Chunk {ans['metadata']['chunk_id']}:\n"
                    f"Findings: {ans['analysis']['key_findings']}\n"
                    f"Missing: {ans['analysis']['missing_info']}"
                    for ans in self.partial_answers
                )

                prompt = f"""Evaluate these analyses about: {self.question}
                
                Current Analysis:
                {context}
                
                Required JSON response:
                {{
                    "status": "complete" or "incomplete",
                    "answer": "summary text" or null,
                    "requested_chunks": [list, of, ids] or null,
                    "confidence": 0.0-1.0
                }}"""

                async with self.session.post(
                    "https://api.groq.com/openai/v1/chat/completions",
                    headers={"Authorization": f"Bearer {self.api_key}"},
                    json={
                        "model": "qwen-qwq-32b",
                        "messages": [{
                            "role": "system",
                            "content": "You must provide valid JSON. Double-check your output before returning."
                        }, {
                            "role": "user",
                            "content": prompt
                        }],
                        "temperature": 0.2,
                        "response_format": {"type": "json_object"}
                    },
                    timeout=25
                ) as response:
                    response.raise_for_status()
                    data = await response.json()
                    decision = json.loads(data['choices'][0]['message']['content'])
                    
                    print(f"Central Agent Decision: {decision}")
                    
                    # Validate central agent response
                    if not all(k in decision for k in ["status", "answer", "requested_chunks", "confidence"]):
                        raise ValueError("Invalid central agent response format")
                    
                    if decision["status"] == "complete":
                        return {
                            "status": "success",
                            "answer": decision["answer"],
                            "confidence": decision["confidence"],
                            "used_chunks": [ans['metadata']['chunk_id'] for ans in self.partial_answers]
                        }
                    
                    # Process requested chunks if incomplete
                    if decision["requested_chunks"]:
                        await self.process_additional_chunks(decision["requested_chunks"])
                        
            except json.JSONDecodeError as e:
                print(f"JSON Error (attempt {attempt + 1}): {str(e)}")
            except Exception as e:
                print(f"Central agent error (attempt {attempt + 1}): {str(e)}")
            
        return {"status": "error", "message": "Failed after maximum attempts"}

    async def process_additional_chunks(self, chunk_ids: List[int]):
        """Processa chunks adicionais solicitados pelo central agent"""
        tasks = []
        for chunk_id in chunk_ids:
            if chunk_id not in self.processed_chunks and 0 <= chunk_id < len(self.all_chunks):
                tasks.append(self.expert_agent(self.all_chunks[chunk_id], chunk_id))
        
        if tasks:
            await asyncio.gather(*tasks)

async def analyze_with_feedback(api_key: str, question: str, chunks: List[str]):
    """Pipeline com loop iterativo de análise"""
    async with SmartAgentSystem(api_key, question, chunks) as system:
        # Primeira rodada com 3 chunks iniciais
        initial_batch = min(3, len(chunks))
        initial_chunk_ids = list(range(initial_batch))
        await system.process_additional_chunks(initial_chunk_ids)
        
        # Run central analysis
        result = await system.central_agent()
        
        # If initial attempt failed but we have partial answers
        if result["status"] == "error" and system.partial_answers:
            return {
                "status": "partial",
                "answers": system.partial_answers,
                "message": "Using partial results after central agent failure"
            }
        return result

# Exemplo de uso:
async def main():
    question = """
    
    Analyze the author's journey through Veridia and Eldoria, detailing the methods they employ
    to document their experiences, and discuss how their interactions with people
    and places shape their understanding of these regions' unique identities.
    """
    result = await analyze_with_feedback(GROQ_API_KEY, question, chunks)
    
    print("\n=== Resultado Final ===")
    print(json.dumps(result, indent=2, ensure_ascii=False))

# Executar no Jupyter:
await main()

Experto Agent Decision: {'analysis': {'key_findings': ["The author documents experiences through sensory descriptions (e.g., Zelphar stew's taste, Bluefire Opal's visual appeal) and participatory engagement with local customs.", "Interactions with locals (e.g., sharing meals, observing miners, attending traditions) reveal Veridia's cultural warmth and Eldoria's reverence for natural resources and communal events.", "The author's journey highlights regional identities through contrasts: Veridia's focus on communal comfort and Eldoria's celebration of physical skill and natural wonders (implied by the javelin contest and Bluefire Opal mining).", "Documentation methods include firsthand participation, observational notes on labor (miners' work) and cultural practices (festivals/contests), and sensory immersion in local cuisine."], 'missing_info': ["Incomplete mention of Eldoria's event ('a...' cut off) limits understanding of its cultural significance.", 'No explicit mention of tools/meth

In [15]:
len(chunks)

5

In [27]:
import aiohttp
import asyncio
import json
from typing import List, Dict, Optional, Tuple

class SmartAgentSystem:
    def __init__(self, api_key: str, question: str, all_chunks: List[str]):
        self.api_key = api_key
        self.question = question
        self.all_chunks = all_chunks
        self.processed_chunks = set()
        self.partial_answers = []
        self.session = None

    async def __aenter__(self):
        self.session = aiohttp.ClientSession()
        return self

    async def __aexit__(self, *args):
        await self.session.close()

    async def expert_agent(self, chunk: str, chunk_id: int) -> Optional[Dict]:
        """Agente especialista com identificação de chunk"""
        try:
            prompt = f"""Analyze this text regarding: {self.question}
            
            Required JSON format:
            {{
                "analysis": {{
                    "key_findings": ["list", "of", "findings"],
                    "missing_info": ["list", "of", "gaps"]
                }},
                "metadata": {{
                    "chunk_id": {chunk_id},
                    "relevance": 0.0-1.0,
                    "needs_more_context": true/false
                }}
            }}

            Text: {chunk}"""

            async with self.session.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={"Authorization": f"Bearer {self.api_key}"},
                json={
                    "model": "qwen-qwq-32b",
                    "messages": [{
                        "role": "system",
                        "content": "You must respond in valid JSON format exactly as specified."
                    }, {
                        "role": "user",
                        "content": prompt
                    }],
                    "temperature": 0.1,
                    "response_format": {"type": "json_object"}
                },
                timeout=20
            ) as response:
                response.raise_for_status()
                data = await response.json()
                result = json.loads(data['choices'][0]['message']['content'])
                
                print(f"Expert Agent Decision: {result}")
                
                # Validate response structure
                required_keys = {'analysis', 'metadata'}
                if not all(key in result for key in required_keys):
                    raise ValueError("Missing required keys in response")
                
                self.processed_chunks.add(chunk_id)
                if result['metadata']['relevance'] > 0.4:
                    self.partial_answers.append(result)
                return result
            
        except json.JSONDecodeError as e:
            print(f"JSON Error in chunk {chunk_id}: {str(e)}")
        except KeyError as e:
            print(f"Missing key in chunk {chunk_id} response: {str(e)}")
        except Exception as e:
            print(f"Error processing chunk {chunk_id}: {str(e)}")
        return None

    async def central_agent(self) -> Dict:
        """Agente central que pode solicitar mais chunks"""
        max_attempts = 1
        print(f"Partial Answers: {self.partial_answers}")
    
        try:
            context = "\n".join(
                f"Chunk {ans['metadata']['chunk_id']}:\n"
                f"Findings: {ans['analysis']['key_findings']}\n"
                f"Missing: {ans['analysis']['missing_info']}"
                for ans in self.partial_answers
            )
            # context = "\n".join(
            #     f"Chunk {ans['metadata']['chunk_id']}:\n"
            #     f"Findings: {ans['analysis']['key_findings']}"
                
            #     for ans in self.partial_answers
            # )

            prompt = f"""Evaluate these analyses about: {self.question}
            
            Current Analysis:
            {context}
            
            Required JSON response:
            {{
                "status": "complete" or "incomplete",
                "answer": "summary text" or null,
                "requested_chunks": [list, of, ids] or null,
                "confidence": 0.0-1.0
            }}"""

            async with self.session.post(
                "https://api.groq.com/openai/v1/chat/completions",
                headers={"Authorization": f"Bearer {self.api_key}"},
                json={
                    "model": "qwen-qwq-32b",
                    "messages": [{
                        "role": "system",
                        "content": "You must provide valid JSON. Double-check your output before returning."
                    }, {
                        "role": "user",
                        "content": prompt
                    }],
                    "temperature": 0.2,
                    "response_format": {"type": "json_object"}
                },
                timeout=25
            ) as response:
                response.raise_for_status()
                data = await response.json()
                decision = json.loads(data['choices'][0]['message']['content'])
                
                print(f"Central Agent Decision: {decision}")
                
                # Validate central agent response
                if not all(k in decision for k in ["status", "answer", "requested_chunks", "confidence"]):
                    raise ValueError("Invalid central agent response format")
                
                if decision["status"] == "complete" or decision['confidence'] > 0.6:
                    return {
                        "status": "success",
                        "answer": decision["answer"],
                        "confidence": decision["confidence"],
                        "used_chunks": [ans['metadata']['chunk_id'] for ans in self.partial_answers]
                    }
                
                else:
                    await self.process_additional_chunks(decision["requested_chunks"])
                    
        except json.JSONDecodeError as e:
            print(f"JSON Error: {str(e)}")
        except Exception as e:
            print(f"Central agent error: {str(e)}")
        

    async def process_additional_chunks(self, chunk_ids: List[int], first=False):
        """Processa chunks adicionais de forma simples e eficiente"""
        # Filtra apenas chunks não processados e válidos
        
        chunks_to_process = self.all_chunks[:2]
        
        # Processa todos os chunks válidos de uma vez
        if chunks_to_process:
            print(f"\nChunks to Process: {len(chunks_to_process)}")
            print("Parte de um Chunk:")
            _ = [chunk[:30] for chunk_id, chunk in enumerate(chunks_to_process)]
            print(_[:30])
            tasks = [
                self.expert_agent(chunk, chunk_id) 
                for chunk_id, chunk in enumerate(chunks_to_process)
            ]
            self.all_chunks = self.all_chunks[-1:]
            print(f"All Chunks: {self.all_chunks[:50]}")
            await asyncio.gather(*tasks)

async def analyze_with_feedback(api_key: str, question: str, chunks: List[str]):
    """Versão simplificada do pipeline de análise"""
    async with SmartAgentSystem(api_key, question, chunks) as system:
        # Processa os 3 primeiros chunks inicialmente
        await system.process_additional_chunks([0, 1, 2], first=True)
        
        # Obtém a decisão do agente central
        
        result = await system.central_agent()
        
        while not result:
            result = await system.central_agent()
        
        print(f"Result: {result}")
        
        # Retorna o resultado seja qual for o status
        return {
            "status": result["status"],
            "answer": result.get("answer"),
            "used_chunks": list(system.processed_chunks)
        }
        
            

# Exemplo de uso:
async def main():
    question = """
    Based on the diary entries, describe the uses of moonpetal by the people of Oakhaven and what the prevalence of this flower suggests about the local culture and economy.
    """
    result = await analyze_with_feedback(GROQ_API_KEY, question, chunks)
    
    print("\n=== Resultado Final ===")
    print(json.dumps(result, indent=2, ensure_ascii=False))

# Executar no Jupyter:
await main()


Chunks to Process: 2
Parte de um Chunk:
['2nd Day of Emberglow 1855 - Ar', '4th Day of Emberglow 1855 - Ex']
All Chunks: ["28th Day of Frostfall 1855 - Oakhaven's Delights\nMy last day in Oakhaven was spent in the town square. I was lucky enough to witness a small festival celebrating the moonpetal harvest. The townspeople were adorned with moonpetal garlands, and the air was filled with music. I finally got to try the famous moonpetal tea. It has a delicate, sweet flavor with a slight floral aroma. I also learned that, beyond its use in tea, moonpetal is used to make a surprisingly potent liquor and that its leaves are used in traditional Oakhaven cooking. I purchased a small vial of moonpetal perfume to take with me, a reminder of this unique place."]
Expert Agent Decision: {'analysis': {'key_findings': ["The diary entry does not mention 'moonpetal' or any references to Oakhaven's cultural or economic practices related to it.", 'The text focuses on coastal exploration, fishing boats

In [116]:
len(chunks)

3

In [106]:
pprint(chunks[2])

('28th Day of Frostfall 1855 - A Celebration of Craft\n'
 'The soft luminescence of dawn cast a gentle glow over the countryside as I '
 "departed for the Veridian Artisans' Fair. [cite: 612, 613] This renowned "
 'event thrives annually, lauded for its display of exceptional craftsmanship, '
 'and I have long anticipated witnessing the skill and creativity of Veridia’s '
 'artisans firsthand. [cite: 613, 614]\n'
 'As I strolled among the bustling stalls, each exhibit boasted a distinct '
 'piece of culture—a tapestry woven with threads of history, or a sculpture '
 'hewn from the heart of the land itself. [cite: 614, 615] An elder craftsman, '
 'his hands worn yet graceful, walked me through his collection of glistening '
 'glassware. [cite: 615, 616, 617] He spoke with pride of his lineage, each '
 'piece carrying a story not just of his making, but of all the hands that '
 'came before him. [cite: 616, 617] This passing of art through generations '
 'mirrors the lasting beauty of th